In [ ]:
!pip install elasticsearch[async] pandasticsearch

In [ ]:
import pandas as pd

from kaggle_secrets import UserSecretsClient
from elasticsearch import Elasticsearch, helpers
from pandasticsearch import Select

In [ ]:
user_secrets = UserSecretsClient()

ELASTICSEARCH_URL = user_secrets.get_secret('ELASTICSEARCH_URL')

In [ ]:
metadata_df = pd.read_csv('/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/metadata.csv')

In [ ]:
metadata_df.head()

In [ ]:
metadata_df.info()

In [ ]:
metadata_records = metadata_df[['cord_uid', 'title', 'abstract']].fillna('').to_dict(orient='records')

In [ ]:
es = Elasticsearch(ELASTICSEARCH_URL)

In [ ]:
# es.indices.delete(index='trec_covid', ignore=[400, 404])
# es.indices.create(index='trec_covid')

In [ ]:
# helpers.bulk(es, metadata_records, index='trec_covid', doc_type='metadata')

In [ ]:
Select.from_dict(es.search(index='trec_covid')).to_pandas()

In [ ]:
topics_rnd3_df = pd.read_csv('/kaggle/input/trec-covid-information-retrieval/topics-rnd3.csv')

In [ ]:
topics_rnd3_df.head()

In [ ]:
topics_rnd3_df.info()

In [ ]:
submission = []
for _, row in topics_rnd3_df.iterrows():
    result_search = es.search(index='trec_covid', query={
        'multi_match': {
            'query': row['query'],
            'fields': ['title', 'abstract']
        }
    })
    result_search_df = Select.from_dict(result_search).to_pandas()
    submission.extend((row['topic-id'], cord_uid) for cord_uid in result_search_df['cord_uid'])
submission_df = pd.DataFrame(submission, columns=('topic-id', 'cord-id'))

In [ ]:
submission_df.head()

In [ ]:
submission_df.info()

In [ ]:
submission_df.to_csv('submission.csv', index=False)